Author : Mounish Juvvadi (ALL THE CELLS)

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

In [35]:
df = pd.read_csv("/content/credit_risk_dataset.csv")

In [36]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [39]:
y = df["loan_status"]
x = df.drop(columns = ["loan_status"])
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=50, train_size=0.80)

In [40]:
y_train.isna().sum()

0

In [41]:
y_test.isna().sum()

0

In [42]:
X_dummies_train = pd.get_dummies(X_train)
X_dummies_test = pd.get_dummies(X_test)

In [43]:
X_dummies_train['person_emp_length'].fillna(method = 'ffill', inplace = True)
X_dummies_train['loan_int_rate'].fillna(method = 'ffill', inplace = True)

In [44]:
X_dummies_train.isna().sum()

person_age                        0
person_income                     0
person_emp_length                 0
loan_amnt                         0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
person_home_ownership_MORTGAGE    0
person_home_ownership_OTHER       0
person_home_ownership_OWN         0
person_home_ownership_RENT        0
loan_intent_DEBTCONSOLIDATION     0
loan_intent_EDUCATION             0
loan_intent_HOMEIMPROVEMENT       0
loan_intent_MEDICAL               0
loan_intent_PERSONAL              0
loan_intent_VENTURE               0
loan_grade_A                      0
loan_grade_B                      0
loan_grade_C                      0
loan_grade_D                      0
loan_grade_E                      0
loan_grade_F                      0
loan_grade_G                      0
cb_person_default_on_file_N       0
cb_person_default_on_file_Y       0
dtype: int64

In [45]:
X_dummies_test['person_emp_length'].fillna(method = 'ffill', inplace = True)
X_dummies_test['loan_int_rate'].fillna(method = 'ffill', inplace = True)
X_dummies_test.isna().sum()

person_age                        0
person_income                     0
person_emp_length                 0
loan_amnt                         0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
person_home_ownership_MORTGAGE    0
person_home_ownership_OTHER       0
person_home_ownership_OWN         0
person_home_ownership_RENT        0
loan_intent_DEBTCONSOLIDATION     0
loan_intent_EDUCATION             0
loan_intent_HOMEIMPROVEMENT       0
loan_intent_MEDICAL               0
loan_intent_PERSONAL              0
loan_intent_VENTURE               0
loan_grade_A                      0
loan_grade_B                      0
loan_grade_C                      0
loan_grade_D                      0
loan_grade_E                      0
loan_grade_F                      0
loan_grade_G                      0
cb_person_default_on_file_N       0
cb_person_default_on_file_Y       0
dtype: int64

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_dummies_train)
X_train_scaled = scaler.transform(X_dummies_train)
X_test_scaled = scaler.transform(X_dummies_test)

In [47]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)
results = model.predict(X_test_scaled)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cf_matrix = confusion_matrix(y_test, results)
cf_matrix_df = pd.DataFrame(
    cf_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [55]:
print("Confusion Matrix")
print(cf_matrix_df)

Confusion Matrix
          Predicted 0  Predicted 1
Actual 0         3213         1916
Actual 1          280         1108


In [57]:
accuracy = accuracy_score(y_test, results)
print("Accuracy Score : ", accuracy)
print("Classification Report")
print(classification_report(y_test, results))

Accuracy Score :  0.6630351388675771
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.63      0.75      5129
           1       0.37      0.80      0.50      1388

    accuracy                           0.66      6517
   macro avg       0.64      0.71      0.62      6517
weighted avg       0.80      0.66      0.69      6517

